In [ ]:
import argparse
import os
import numpy as np
import math

import torchvision.transforms as transforms
from torchvision.utils import save_image

from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable

import torch.nn as nn
import torch.nn.functional as F
import torch

import pandas

In [ ]:
n_epochs = 1000
batch_size = 128
lr = 0.001

b1 = 0.5
b2 = 0.999

n_cpu = 8
latent_dim = 20

img_size = 28
channels = 1
sample_interval = 400

In [ ]:
from torch.utils.data import Dataset

class PresetDataset(Dataset):
    
    def __init__(self,filepath):
        self.data = pandas.read_csv(filepath)
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self,idx):
        return torch.FloatTensor(list(self.data.iloc[idx]))

In [ ]:
preset_dataset = PresetDataset("../presets_all_norm_minmax_one_hot_neg.csv")
preset_dataset[101].size()

In [ ]:
data_size = preset_dataset[101].size()[0]

In [ ]:
data_size

In [ ]:
img_shape = (channels, img_size, img_size)

In [ ]:
cuda = True if torch.cuda.is_available() else False

In [ ]:
print(data_size)

In [ ]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()

        def block(in_feat, out_feat, normalize=True):
            layers = [nn.Linear(in_feat, out_feat)]
            if normalize:
                layers.append(nn.BatchNorm1d(out_feat,0.8)) #0.8
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers

        self.model = nn.Sequential(
            *block(latent_dim, 128, normalize=False),
            *block(128,256),
            *block(256, 512),
            *block(512, 128),
            nn.Linear(128, data_size),
            nn.Tanh()
        )

    def forward(self, z):
        img = self.model(z)
        #img = img.view(img.size(0), data_size)
        return img

In [ ]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        self.model = nn.Sequential(
            nn.Linear(data_size, 64),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(64, 32),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(32, 1),
            nn.Sigmoid(),
        )

    def forward(self, img):
        #img_flat = img.view(img.size(0), -1)
        img_flat = img
        validity = self.model(img_flat)

        return validity

In [ ]:
# Loss function
adversarial_loss = torch.nn.BCELoss()

# Initialize generator and discriminator
generator = Generator()
discriminator = Discriminator()

if cuda:
    generator.cuda()
    discriminator.cuda()
    adversarial_loss.cuda()

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [ ]:
count_parameters(generator)

In [ ]:
#os.makedirs("./PyTorch-GAN/data/mnist", exist_ok=True)
dataloader = torch.utils.data.DataLoader(
    preset_dataset,
    batch_size=batch_size,
    shuffle=True,
)

In [ ]:
#for i, batch in enumerate(dataloader):
#        print(batch)

In [ ]:
# Optimizers
optimizer_G = torch.optim.Adam(generator.parameters(), lr=lr, betas=(b1, b2))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=lr, betas=(b1, b2))

Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor

In [ ]:
import csv

def save_sample(data,i,num=8):
    output = data.cpu().detach().numpy()
    #print(output[0])
    name = "./out_onehot/"+str(i)+".csv"
    header_row = ["x"]*data_size
    with open(name, "w") as f:
        writer = csv.writer(f)
        writer.writerow(header_row)
        writer.writerows(output)

In [ ]:
# ----------
#  Training
# ----------
master_num = 0
for epoch in range(n_epochs):
    for i, imgs in enumerate(dataloader):
        #print(imgs)

        # Adversarial ground truths
        valid = Variable(Tensor(imgs.size(0), 1).fill_(1.0), requires_grad=False)
        fake = Variable(Tensor(imgs.size(0), 1).fill_(0.0), requires_grad=False)

        # Configure input
        real_imgs = Variable(imgs.type(Tensor))

        # -----------------
        #  Train Generator
        # -----------------

        optimizer_G.zero_grad()

        # Sample noise as generator input
        z = Variable(Tensor(np.random.normal(0, 1, (imgs.shape[0], latent_dim))))

        # Generate a batch of images
        gen_imgs = generator(z)

        # Loss measures generator's ability to fool the discriminator
        g_loss = adversarial_loss(discriminator(gen_imgs), valid)

        g_loss.backward()
        optimizer_G.step()

        # ---------------------
        #  Train Discriminator
        # ---------------------

        optimizer_D.zero_grad()

        # Measure discriminator's ability to classify real from generated samples
        real_loss = adversarial_loss(discriminator(real_imgs), valid)
        fake_loss = adversarial_loss(discriminator(gen_imgs.detach()), fake)
        d_loss = (real_loss + fake_loss) / 2

        d_loss.backward()
        optimizer_D.step()

        batches_done = epoch * len(dataloader) + i
        if batches_done % sample_interval/2 == 0:
            print(
                "[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G loss: %f]"
                % (epoch, n_epochs, i, len(dataloader), d_loss.item(), g_loss.item())
            )
            save_sample(gen_imgs,master_num)
            master_num+=1

        
        #if batches_done % sample_interval == 0:
        #    save_image(gen_imgs.data[:25], "images/%d.png" % batches_done, nrow=5, normalize=True)